In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [38]:
import pandas as pd

#Since we have used pickle
data = pd.read_pickle('preprocessed_data.pkl')

In [39]:
# Load your data
data = pd.read_csv('Dataset/CSV FILE/Total_objects_dataset_Uncleaned_.csv')

In [40]:
# Attempt to convert 'LAUNCH' to datetime, handling errors and mixed formats
data['LAUNCH'] = pd.to_datetime(data['LAUNCH'], errors='coerce', dayfirst=True)

# Extract the year from the 'LAUNCH' column
data['YEAR'] = data['LAUNCH'].dt.year


C:\Users\anura\AppData\Local\Temp\ipykernel_18672\3939401529.py:2: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data['LAUNCH'] = pd.to_datetime(data['LAUNCH'], errors='coerce', dayfirst=True)


In [41]:
# Handle missing values in 'YEAR' if any remain
data['YEAR'].fillna(data['YEAR'].median(), inplace=True)

In [42]:
# Continue with the rest of our preprocessing
numerical_cols = ['PERIOD', 'INCLINATION', 'APOGEE', 'PERIGEE', 'RCSVALUE', 'YEAR']
for col in numerical_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col].fillna(data[col].median(), inplace=True)

In [43]:
# Encode categorical variables
categorical_cols = ['OBJECT_ID', 'OBJECT_NAME', 'NORAD_CAT_ID', 'COUNTRY']
for col in categorical_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

In [44]:
# Prepare data for machine learning: Features and target variable
features = numerical_cols + categorical_cols
X = data[features]
y = data['RCS_SIZE'].astype(str) 

In [45]:
#Handling missing 'RCS_SIZE' values using RandomForest Classifier
# First, encode 'RCS_SIZE' for use in classification
y = LabelEncoder().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [47]:
# Ensure 'X' is aligned correctly with the original DataFrame's indexing
X.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)

# Re-evaluate missing indices after any other data manipulations
missing_indices = data[data['RCS_SIZE'].isnull()].index

# Predict and fill missing 'RCS_SIZE' only if there are any missing
if not missing_indices.empty:
    predicted_rcs = clf.predict(X.loc[missing_indices])
    # Decode predictions using the LabelEncoder instance used to fit the data
    le = LabelEncoder().fit(data['RCS_SIZE'].astype(str))  # Make sure to fit on non-null values if necessary
    data.loc[missing_indices, 'RCS_SIZE'] = le.inverse_transform(predicted_rcs)
else:
    print("No missing 'RCS_SIZE' values to predict.")

# Proceed with the rest of your processing


In [49]:
# Make sure 'X' is defined correctly with all expected features
X = data[features]

# Identify missing indices in 'RCS_SIZE'
missing_indices = data[data['RCS_SIZE'].isnull()].index

# Verify if there are indeed missing indices
if not missing_indices.empty:
    # Select the subset of X corresponding to missing 'RCS_SIZE'
    X_missing = X.loc[missing_indices]

    # Verify if X_missing is not empty and contains the right amount of columns
    if not X_missing.empty and X_missing.shape[1] == len(features):
        # Ensure X_missing is in the correct format (if required, recheck data types or reset index)
        X_missing.reset_index(drop=True, inplace=True)

        # Predict missing 'RCS_SIZE' values using the classifier
        predicted_rcs = clf.predict(X_missing)

        # Decode predictions back to original labels if needed
        le = LabelEncoder()
        le.fit(data['RCS_SIZE'].dropna().astype(str))  # Fit label encoder on non-missing RCS_SIZE
        data.loc[missing_indices, 'RCS_SIZE'] = le.inverse_transform(predicted_rcs)

        print("Missing 'RCS_SIZE' values have been successfully predicted and filled.")
    else:
        print(f"Error: The subset of X for missing indices is empty or incorrectly formatted. Shape: {X_missing.shape}")
else:
    print("No missing 'RCS_SIZE' values to predict.")


No missing 'RCS_SIZE' values to predict.


In [52]:
# Reset the index of X to ensure alignment
X.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)

# Re-evaluate missing indices to ensure they are accurate
missing_indices = data[data['RCS_SIZE'].isnull()].index

# Debugging print to check the content and size of missing_indices
print("Missing indices:", missing_indices)

if len(missing_indices) > 0:
    # Extract the rows from X corresponding to missing_indices
    X_missing = X.loc[missing_indices]

    # Check if X_missing is non-empty and contains the expected number of features
    if not X_missing.empty and X_missing.shape[1] == X.shape[1]:
        # Predict missing 'RCS_SIZE' values
        predicted_rcs = clf.predict(X_missing)

        # Decode predictions using the previously fitted LabelEncoder
        le = LabelEncoder().fit(data['RCS_SIZE'].dropna().astype(str))
        data.loc[missing_indices, 'RCS_SIZE'] = le.inverse_transform(predicted_rcs)

        print("Missing 'RCS_SIZE' values predicted and filled.")
    else:
        print("Error: No data to predict. Check X_missing formation:", X_missing)
else:
    print("No missing 'RCS_SIZE' values found for prediction.")


Missing indices: Index([], dtype='int64')
No missing 'RCS_SIZE' values found for prediction.


In [53]:
# Check initial state of RCS_SIZE for missing values
initial_missing_count = data['RCS_SIZE'].isnull().sum()
print(f"Initial missing RCS_SIZE values: {initial_missing_count}")

# If there are missing values, continue with the planned processing
if initial_missing_count > 0:
    # Predict and fill missing 'RCS_SIZE'
    missing_indices = data[data['RCS_SIZE'].isnull()].index
    predicted_rcs = clf.predict(X[missing_indices])
    data.loc[missing_indices, 'RCS_SIZE'] = LabelEncoder().inverse_transform(predicted_rcs)  # Decode predictions
    pass
else:
    print("No initial missing values in RCS_SIZE. Check data loading and preprocessing steps.")


Initial missing RCS_SIZE values: 0
No initial missing values in RCS_SIZE. Check data loading and preprocessing steps.


In [59]:
# Correctly loading a pickle file without additional arguments
data = pd.read_pickle('preprocessed_data.pkl')

# Re-check for missing values
initial_missing_count = data['RCS_SIZE'].isnull().sum()
print(f"Initial missing RCS_SIZE values: {initial_missing_count}")


KeyError: 'RCS_SIZE'

In [60]:
# Print all column names in the DataFrame
print(data.columns)


Index(['OBJECT_ID', 'OBJECT_NAME', 'NORAD_CAT_ID', 'PERIOD', 'INCLINATION',
       'APOGEE', 'PERIGEE', 'RCSVALUE', 'LAUNCH', 'COMMENT',
       ...
       'COUNTRY_URY', 'COUNTRY_US', 'COUNTRY_USBZ', 'COUNTRY_VENZ',
       'COUNTRY_VTNM', 'RCS_SIZE_LARGE', 'RCS_SIZE_MEDIUM', 'RCS_SIZE_SMALL',
       'ORBITAL_RANGE', 'TIME_SINCE_LAUNCH'],
      dtype='object', length=118)


In [63]:
# Loading the pickle file
data = pd.read_csv('Dataset/CSV FILE/Total_objects_dataset_Uncleaned_.csv')

# Check if 'RCS_SIZE' is in the DataFrame
if 'RCS_SIZE' in data.columns:
    initial_missing_count = data['RCS_SIZE'].isnull().sum()
    print(f"Initial missing RCS_SIZE values: {initial_missing_count}")
else:
    print("Column 'RCS_SIZE' does not exist in the DataFrame. Available columns are:", data.columns)


Initial missing RCS_SIZE values: 1285
